<a href="https://colab.research.google.com/github/Winest-Nigeria/fgan-I-298-R1/blob/main/2023/UsecaseDocToNodes_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install PyPDF2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.8 MB/s eta 0:00:00


In [ ]:
import PyPDF2
import csv

# Open the PDF file in read-binary mode
with open('example.pdf', 'rb') as pdf_file:
    # Create a PDF reader object
    pdf_reader = PyPDF2.PdfFileReader(pdf_file)

    # Get the first page of the PDF document
    first_page = pdf_reader.getPage(0)

    # Extract the text from the first page
    text = first_page.extractText()

    # Define the name of the CSV file and the column header
    csv_file = 'example.csv'
    header = ['text']

    # Write the text to the CSV file
    with open(csv_file, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(header)
        writer.writerow([text])


In [ ]:
#!pip install argilla
%pip install "argilla[server, listeners]==1.1.1"

In [ ]:
import argilla as rg

In [ ]:
%pip install "colab-xterm~=0.1.2" # since colab doesn't have terminal, this creates a terminal within colab to be used to run ngrok 
%pip install "pyngrok~=5.2.1" # ngrok python library

In [ ]:
%%bash

wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.10.2-linux-x86_64.tar.gz -q
tar -xzf elasticsearch-7.10.2-linux-x86_64.tar.gz
chown -R daemon:daemon elasticsearch-7.10.2

In [ ]:
%%bash --bg

sudo -u daemon -- elasticsearch-7.10.2/bin/elasticsearch

In [ ]:
import time
time.sleep(30)  # sleeping to give ES time to set up. Otherwise downstream code will bug

In [ ]:
# create a terminal to run Argilla with, in case you don't have Colab Pro.
# type "python -m argilla" into the terminal that appears below this code cell.
%load_ext colabxterm
%xterm

In [ ]:
import getpass

In [ ]:
from pyngrok import ngrok, conf

print("Enter your authtoken, which can be copied from https://dashboard.ngrok.com/auth")
print("You need to create a free ngrok account to get an authtoken. The token looks something like this: ASDO1283YZaDu95vysXYIUXZXYRR_54YfASDIb8cpNfVoz349587")
conf.get_default().auth_token = getpass.getpass()
# if the above does not work, you can try: 
#ngrok.set_auth_token("<INSER_YOUR_NGROK_AUTHTOKEN>")

In [ ]:
# disconnect all existing tunnels to avoid issues when rerunning cells
[ngrok.disconnect(tunnel.public_url) for tunnel in ngrok.get_tunnels()]

# create the public link
# ! check whether this is actually the localhost port Argilla is running on via the terminal above 
ngrok_tunnel = ngrok.connect(6900)  # insert the port number Argilla is running on. e.g. 6900 if the terminal displays something like "Uvicorn running on http://0.0.0.0:6900"
print("You can now access the Argilla localhost with the public link below. (It should look something like 'http://X03b-34-XXX-237-25.ngrok.io')\n")
print(f"Your ngrok public link: {ngrok_tunnel}\n")
print("After clicking on the link, there will be a warning, which you can ignore")
print("You can then login with the default agrilla username 'argilla' and password '1234'")

In [ ]:
!pip install pandas
import pandas as pd
import argilla as rg
# Read the CSV file into a pandas DataFrame
#dataframe = pd.read_csv("/content/sample_data/kaffee_reviews.csv")
dataframe = pd.read_csv("/content/sample_data/use-case-og-001-1.csv")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
dataset = dataset.rename_column("relation", "text")


In [ ]:
# pip install -U spacy
# python -m spacy download en_core_web_sm
import spacy

# Load a german spaCy model to tokenize our text
nlp = spacy.load("en_core_web_sm")

# Define our tokenize function
def tokenize(row):
    tokens = [token.text for token in nlp(row["text"])]
    return {"tokens": tokens}


# Map the tokenize function to our dataset
dataset = dataset.map(tokenize)

Map:   0%|          | 0/15 [00:00<?, ? examples/s]

In [ ]:
dataset.select(range(3)).to_pandas()


,Actor,Description,text,tokens
0,KB,Stores knowledge related to the AN.,x -> refer -> KB,"[x, -, >, refer, -, >, KB]"
1,None,Can be referred for reports or translation of ...,KB -> export -> KB,"[KB, -, >, export, -, >, KB]"
2,None,Can export parts of the KB.,KB -> import -> KB,"[KB, -, >, import, -, >, KB]"


In [ ]:
import argilla as rg

# Read Dataset into a Argilla Dataset
dataset_rg = rg.read_datasets(dataset, task="TokenClassification")


In [ ]:
# Log the dataset to the Argilla web app
rg.log(dataset_rg, "coffee-reviews")


  0%|          | 0/15 [00:00<?, ?it/s]

15 records logged to http://localhost:6900/datasets/argilla/coffee-reviews


BulkResponse(dataset='coffee-reviews', processed=15, failed=0)

In [ ]:


# Create a basic token classification record
record = rg.TokenClassificationRecord(
    text="OthnyEl is a professor at Harvard",
    tokens=["OthnyEl", "is", "a", "professor", "at", "Harvard"],
    prediction=[("NAME", 0, 7), ("LOC", 26, 33)],
)
rg.log(record, "my_token_record")



  0%|          | 0/1 [00:00<?, ?it/s]

1 records logged to http://localhost:6900/datasets/argilla/my_token_record


BulkResponse(dataset='my_token_record', processed=1, failed=0)